# DSCI 563 - Unsupervised Learning

# Lab 1: Clustering 

## Imports <a name="im"></a>

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN, KMeans
from sklearn.feature_extraction.text import CountVectorizer

%matplotlib inline
pd.set_option("display.max_colwidth", 0)

<br><br><br><br>

## Submission instructions <a name="si"></a>
<hr>
rubric={mechanics:3}

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **Please add a link to your GitHub repository here: LINK TO YOUR  REPO**
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).
- Make at least three commits in your lab's GitHub repository.
- Push the final .ipynb file with your solutions to your GitHub repository for this lab.
- Make sure to enroll to Gradescope via [Canvas](https://canvas.ubc.ca/courses/83554).
- Upload the .ipynb file to Gradescope.
- If the .ipynb file is too big or doesn't render on Gradescope for some reason, also upload a pdf or html in addition to the .ipynb. 
- Make sure that your plots/output are rendered properly in Gradescope.

> [Here](https://github.com/UBC-MDS/public/tree/master/rubric) you will find the description of each rubric used in MDS.

> As usual, do not push the data to the repository. 

<br><br><br><br>

## Exercise 1: Document clustering warm-up <a name="1"></a>
<hr>

In lecture 1, we explored customer segmentation, a popular application of clustering. In this lab, we will explore another popular application, [**document clustering**](https://en.wikipedia.org/wiki/Document_clustering). A large amount of unlabeled text data is available out there (e.g., news, recipes, online Q&A, tweets), and clustering is a commonly used technique to organize this data in a meaningful way. 

As a warm up, in this exercise, you will cluster sentences from a toy corpus. Later in the lab, you will work with a real corpus. 

**Your tasks:**

Run the code below which 
- extracts content of Wikipedia articles on a set of queries
- tokenizes the text (i.e., separates sentences) and 
- stores the 2nd sentence in each article as a document representing that article

> *Note 1: If you have created `conda` environment using [the course `yml` file](https://github.ubc.ca/mds-2021-22/DSCI_563_unsup-learn_students/blob/master/env-dsci-563.yml), you should have the following packages. If not, you may have to install appropriate packages in our course's environment.*

> *Note 2: Feel free to experiment with Wikipedia queries of your choice. But stick to the provided list for the final submission so that it's easier for the TAs to grade your submission.*

> *Note 3: For tokenization we are using the `nltk` package. Even if you have the package installed via the course `conda` environment, you might have to download `nltk` pre-trained models, which can be done with the code below.*

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
import wikipedia
from nltk.tokenize import sent_tokenize, word_tokenize

queries = [
    "hummus food",
    "bread food",
    "artificial intelligence",
    "unsupervised learning",
    "football sport",
    "ice hockey",
]

wiki_dict = {"wiki query": [], "text": [], "n_words": []}
for i in range(len(queries)):
    text = sent_tokenize(wikipedia.page(queries[i]).content)[1]
    wiki_dict["text"].append(text)
    wiki_dict["n_words"].append(len(word_tokenize(text)))
    wiki_dict["wiki query"].append(queries[i])

wiki_df = pd.DataFrame(wiki_dict)
wiki_df

Our toy corpus has six toy documents (`text` column in the dataframe) extracted from Wikipedia queries. 

<br><br>

### 1.1 How many clusters? 
rubric={reasoning:2}


**Your tasks:**

1. If you are asked to cluster the documents from this toy corpus manually, how many clusters would you identify and how would you label each cluster?   

#### Solution_1_1_1




<br><br>

### 1.2 `KMeans` with bag-of-words representation 
rubric={accuracy:4}

In previous machine learning courses, we have seen that before we pass text to machine learning models, we need to encode it to a numeric representation. We will try two representations: bag-of-words representation and sentence embedding representation. First, let's try our good old friend, bag-of-words.

**Your tasks:**

1. Create bag-of-words representation using [`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) for the `text` column in `wiki_df` above.
2. Cluster the documents with this representation and [`KMeans` clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) with the following arguments: 
    - `random_state=42` (for reproducibility)
    - `n_clusters`=the number of clusters you identified in 1.1

#### Solution_1_2_1


#### Solution_1_2_2


<br><br>

### 1.3 Sentence embedding representation
rubric={accuracy:3}

As we have seen before, bag-of-words representation is limited in that it does not take into account word ordering and context. There are other richer and more expressive representations of text, and we are going to use one such representation in this lab. We will call it **sentence embedding representation**. We'll use [sentence transformer](https://www.sbert.net/index.html) to extract these representations. At this point it's enough to know that this is an alternative representation of text which usually works better than simple bag-of-words representation. We'll learn more about these representations later in this course and in DSCI 575 if we get a chance.

**Your tasks:**

1. Run the code below to create sentence embedding representation of documents in our toy corpus. 
2. Cluster the documents with this representation (`emb_sents`) and `KMeans` with the following arguments: 
    - `random_state=42` (for reproducibility)
    - `n_clusters`=the number of clusters you identified in 1.1

In [ ]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("paraphrase-distilroberta-base-v1")

In [ ]:
emb_sents = embedder.encode(wiki_df["text"])
emb_sent_df = pd.DataFrame(emb_sents, index=wiki_df.index)
emb_sent_df

#### Solution_1_3_2


<br><br>

### 1.4 DBSCAN with cosine distance  
rubric={accuracy:4}

Let's try [`DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) on our toy dataset. K-Means is kind of bound to the Euclidean distance because it is based on the notion of means. With `DBSCAN` we can try different distance metrics. In the context of text, [cosine similarities](https://scikit-learn.org/stable/modules/metrics.html#cosine-similarity) or cosine distances tend to work better. Given vectors $u$ and $v$, the **cosine distance** between the vectors is defined as: 

$$distance_{cosine}(u,v) = 1 - (\frac{u \cdot v}{\left\lVert u\right\rVert_2 \left\lVert v\right\rVert_2})$$


**Your tasks**

1. Cluster documents in our toy corpus encoded with sentence embedding representation (`emb_sents`) and [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html?highlight=dbscan#sklearn.cluster.DBSCAN) with `metric='cosine'`. 

> *Note: You will have to set appropriate values for the hyperparamters `eps` and `min_samples` to get meaningful clusters, as default values for these hyperparameters are unlikely to work on this toy dataset.*

#### Solution_1_4_1


<br><br>

### 1.5 Discussion
rubric={reasoning:4}

**Your tasks:**

1. Reflect and comment on the clusters identified by each of the methods you explored in 1.2, 1.3, and 1.4. Are these methods doing a reasonable job in clustering the sentences in our toy corpus? Do the clustering results change depending upon the representation you use?  

#### Solution_1_5_1



<br><br>

### 1.6 Visualizing clusters
rubric={viz:3}

One thing we can do with unlabeled data is visualizing it. That said, our data is high dimensional and high-dimensional data is hard to visualize. For example, in sentence embedding representation, each example is represented with 768 dimensions. One way to visualize high-dimensional data is by applying dimensionality reduction to get the most important (2 or 3) components of the dataset and visualizing this low-dimensional data. 

Given data as a `numpy` array and corresponding cluster assignments, the `plot_umap_clusters` function below transforms the data by applying dimensionality reduction technique called [UMAP](https://umap-learn.readthedocs.io/en/latest/) to it and plots the transformed data with different colours for different clusters. 

> *Note: At this point we are using this function only for visualization and you are not expected to understand the UMAP part. We'll learn about similar dimensionality reduction techniques in the next two weeks.* 

I forgot to put this package in the course environment file. So to run the code below, you'll have to install the `umap-learn` package in the course conda environment either with `conda` or `pip`, as described in the [documentation](https://umap-learn.readthedocs.io/en/latest/index.html). 

```
> conda activate 563
> conda install -c conda-forge umap-learn
```

or 

```
> conda activate 563
> pip install umap-learn
```


**Your tasks:**

1. Visualize the clusters created by the three methods above using `plot_umap_clusters` function above:
    - KMeans with bag-of-words representation 
    - KMeans with sentence embedding representation
    - DBSCAN with sentence embedding representation 
 

In [ ]:
import umap

In [ ]:
def plot_umap_clusters(
    data,
    cluster_labels,
    raw_sents=wiki_df["text"],
    show_labels=False,
    size=50,
    n_neighbors=15,
    title="UMAP visualization",
    ignore_noise=False,
):
    """
    Carry out dimensionality reduction using UMAP and plot 2-dimensional clusters.

    Parameters
    -----------
    data : numpy array
        data as a numpy array
    cluster_labels : list
        cluster labels for each row in the dataset
    raw_sents : list
        the original raw sentences for labeling datapoints
    show_labels : boolean
        whether you want to show labels for points or not (default: False)
    size : int
        size of points in the scatterplot
    n_neighbors : int
        n_neighbors hyperparameter of UMAP. See the documentation.
    title : str
        title for the visualization plot

    Returns
    -----------
    None. Shows the clusters.
    """

    reducer = umap.UMAP(n_neighbors=n_neighbors)
    Z = reducer.fit_transform(data)  # reduce dimensionality
    umap_df = pd.DataFrame(data=Z, columns=["dim1", "dim2"])
    umap_df["cluster"] = cluster_labels

    if ignore_noise:
        umap_df = umap_df[umap_df["cluster"] != -1]

    labels = np.unique(umap_df["cluster"])

    fig, ax = plt.subplots(figsize=(10, 7))
    ax.set_title(title)

    scatter = ax.scatter(
        umap_df["dim1"],
        umap_df["dim2"],
        c=umap_df["cluster"],
        cmap="tab20b",
        s=size,
        edgecolors="k",
        linewidths=0.1,
    )

    legend = ax.legend(*scatter.legend_elements(), loc="best", title="Clusters")
    ax.add_artist(legend)

    if show_labels:
        x = umap_df["dim1"].tolist()
        y = umap_df["dim2"].tolist()
        for i, txt in enumerate(raw_sents):
            ax.annotate(" ".join(txt.split()[:10]), (x[i], y[i]))
    plt.show()

#### Solution_1_6_1


<br><br><br><br>

## Exercise 2: [Food.com](https://www.food.com/) recipes 

<hr>

Now that we have applied document clustering on a toy dataset, let's cluster sentences from a real corpus. In this lab we will work with a sample of [Kaggle's Food.com recipes corpus](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions). This corpus contains 180K+ recipes and 700K+ recipe reviews. In this lab, we'll only focus on recipes and **not** on recipe reviews. The recipes are present in `RAW_recipes.csv`. Our goal is to find main categories or groupings of recipes based on their names. 

**Your tasks:**
- Download [`RAW_recipes.csv`](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_recipes.csv) and put it in the lab folder. As usual, do not push the CSV in your repository. 
- Run the code below. The dataset is quite large, and in this assignment, for speed, you will work with a sample of the dataset. The function `get_recipes_sample` below carries out some preliminary preprocessing and returns a sample of the recipes with most frequent tags. 

> *Note: Depending upon the capacity of your computer, feel free to increase or decrease the size of this sample by changing the value for `n_tags`.* 

In [ ]:
orig_recipes_df = pd.read_csv("RAW_recipes.csv")
orig_recipes_df.shape

In [ ]:
def get_recipes_sample(orig_recipes_df, n_tags=300, min_len=5):
    orig_recipes_df = orig_recipes_df.dropna()  # Remove rows with NaNs.
    orig_recipes_df = orig_recipes_df.drop_duplicates(
        "name"
    )  # Remove rows with duplicate names.
    # Remove rows where recipe names are too short (< 5 characters).
    orig_recipes_df = orig_recipes_df[orig_recipes_df["name"].apply(len) >= min_len]
    # Only consider the rows where tags are one of the most frequent n tags.
    first_n = orig_recipes_df["tags"].value_counts()[0:n_tags].index.tolist()
    recipes_df = orig_recipes_df[orig_recipes_df["tags"].isin(first_n)]
    return recipes_df

In [ ]:
recipes_df = get_recipes_sample(orig_recipes_df)
recipes_df.shape

In [ ]:
recipes_df["name"]

<br><br>

**In the rest of the lab, we will use `recipes_df` above, which is a subset of the original dataset.** 

<br><br>

### 2.1 Longest and shorted recipe names 
rubric={accuracy:2}

**Your tasks:**

1. Print the shortest and longest recipe names (in terms of number of characters) from `recipes_df`. 

#### Solution_2_1_1


<br><br>

### (optional) 2.2 More EDA
rubric={reasoning:1}

**Your tasks:**
1. Create a word cloud for the recipe names. You can use [the `wordcloud` package](https://github.com/amueller/word_cloud) for this, which you will have to install in the course environment. 
2. Come up with one more EDA question for this dataset and explore the question. 

#### Solution_2_2_1


#### Solution_2_2_2


<br><br>

### 2.3 Representing recipe names
rubric={reasoning:3}

The next step is creating a representation of recipe names. 

**Your tasks:**

1. Similar to Exercise 1, create sentence embedding representation of recipe names (`name` column in `recipes_df`).  For the rest of the lab, we'll stick to the sentence embedding representation of recipe names.



> *If you create a dataframe with sentence embedding representation, set the index to `recipes_df.index` so that the indices match with the indices of the sample we are working with.*  
> *This might take a while to run.*

#### Solution_2_3_1



<br><br><br><br>

## Exercise 3: K-Means on Food.com recipe names <a name="3"></a>
<hr>

### 3.1 Choosing K for K-Means
rubric={viz:6}

For K-Means you need to specify the number of clusters in advance, which is often challenging to do on real datasets. As we saw in the lecture, there is no definitive method to select the number of clusters. That said, there are some approaches which may help us with this process. In this exercise, you'll explore three such approaches. 

**Your tasks:**

1. Consider a reasonable range of K (`n_clusters`) values and visualize the Elbow plot. 
2. Consider a reasonable range of K (`n_clusters`) values and visualize Silhouette plots. 
2. Consider a reasonable range of K (`n_clusters`) and visualize the clusters created by K-Means by using `plot_umap_clusters` function from Exercise 1. 

> You may use the [`yellowbrick`](https://www.scikit-yb.org/en/latest/) package for visualizing the Elbow plot and the Silhouette plots.      

> The range of K or `n_clusters` values does not have to be the same in the three cases above. 

> Use the default value of `show_labels=False` when you call function `plot_umap_clusters`, as we do not want to display labels of thousands of data points.    

#### Solution_3_1_1



#### Solution_3_1_2



#### Solution_3_1_3


<br><br>

### 3.2 Discussion 
rubric={reasoning:4}

**Your tasks:** 
1. Comment on your results from 3.1. Are the plots above useful in narrowing down the range of values for `n_clusters`? Based on these visualizations, what value or a range of values seems reasonable for `n_clusters` in this problem? 

#### Solution_3_2_1



<br><br>

### 3.3 Sampling recipe names from clusters
rubric={accuracy:5}

It's likely that with the methods in the previous exercises you did not get a satisfactory answer on how many clusters should be appropriate for this problem. One of the most important steps in clustering is manual interpretation of clusters. In this exercise, you will examine some samples from different clusters given by K-Means, which might give you a better understanding on the number of clusters and whether the clusters make sense or not.  

**Your tasks:**
1. Based on your answer in Exercise 3.1 and 3.2, pick one or two reasonable values for `n_clusters` and train `KMeans` with those values and `random_state=42`. 
2. Sample some examples (e.g., 10 to 15 recipe names) from each cluster and examine them manually.

#### Solution_3_3_1 and Solution_3_3_2


<br><br>

### 3.4 Manual interpretation of clusters
rubric={reasoning:5}

**Your tasks:**
1. Do you see a clear distinction between clusters? What topics/themes do to see in different clusters? 

#### Solution_3_4_1



<br><br>

### 3.5 Examining other attributes in the data for each cluster 
rubric={accuracy:3,reasoning:2}

Recall that we have other attributes such as `minutes`, `n_steps`, `n_ingredients` in our `recipes_df`. Let's examine these attributes for different clusters. 

**Your tasks:**

1. For each cluster, show average of `minutes`, `n_steps`, `n_ingredients`.
2. Do you see any striking differences in the values of these features between clusters? 

#### Solution_3_5_1


#### Solution_3_5_2

 

<br><br>

### (optional) 3.6 Examine cluster tags 
rubric={reasoning:1}

**Your tasks:**
1. Explore the tags associated with different clusters. Each recipe has a list of tags given in the `Tags` column. You could, for instance, find the most common items in tags for each recipe. 

#### Solution_3_5_1


<br><br><br><br>

## Exercise 4: DBSCAN on Food.com recipe names <a name="4"></a>
<hr>


### 4.1 Choosing hyperparameters `eps` and `min_samples`
rubric={accuracy:6}

In this exercise you will cluster the recipes using `DBSCAN` with sentence embedding representation and cosine distance metric. 

**Your tasks:**

1. Pick reasonable values for the hyperparameters `eps` and `min_samples` of [DBSCAN](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) by examining cluster sizes and visualizing plots created with `plot_umap_clusters`. 
2. Train a DBSCAN clustering model with the chosen values for the hyperparameters. 

> If you want to skip noise points, you can try passing `ignore_noise=False` when you call `plot_umap_clusters`. 

#### Solution_4_1_1



#### Solution_4_1_2


<br><br>

### 4.2 Interpretation of DBSCAN clusters
rubric={accuracy:3,reasoning:3}

In this exercise you will examine some examples from different clusters given by DBSCAN. 

**Your tasks:**
1. Sample some recipe names (e.g., 10 to 15) from each cluster identified by DBSCAN and examine them manually.
3. How many noise points are there? Do you see a clear distinction between the clusters? What topics/themes do you see in different clusters? Are these clusters very different compared to the clusters identified by K-Means? 

#### Solution_4_2_1


#### Solution_4_2_2



<br><br>

### (optional) 4.3 Dendrogram
rubric={reasoning:1}

**Your tasks:**

1. Show a dendrogram with `p=10` and `truncate_mode=level` on sentence embeddings of recipes with average linkage and `metric="cosine"`.
2. Briefly comment on the results.

> *Note: Try orientation="left" of `dendrogram` for better readability of the dendrogram.*

#### Solution_4_3_1 and Solution_4_3_2



<br><br><br><br>

## (optional) Exercise 5: Vector Quantization <a name="4"></a>
<hr>
rubric={reasoning:1}

One more application of clustering is _vector quantization_, where we find a prototype point for each cluster and replace points in the cluster by their prototype. If our inputs are images, vector quantization gives us a rudimentary image compression algorithm.

We will implement image quantization by filling in the `quantize` and `dequantize` functions below. The `quantize` function should take in an image,  and using the pixels as examples and the 3 colour channels as features, run K-Means clustering on the data with $2^b$ clusters for some hyperparameter $b$. The code should store the cluster means and return the cluster assignments. The `dequantize` function should return a version of the image (the same size as the original) where each pixel's original colour is replaced with the nearest prototype colour.

To understand why this is compression, consider the original image space. Say the image can take on the values $0,1,\ldots,255$ in each colour channel. Since $2^8=256$ this means we need 8 bits to represent each colour channel, for a total of 24 bits per pixel. Using our method, we are restricting each pixel to only take on one of $2^b$ colour values. In other words, we are compressing each pixel from a 24-bit colour representation to a $b$-bit colour representation by picking the $2^b$ prototype colours that are "most representative" given the content of the image. 

**Your tasks:**

1. Complete the `quantize` and `dequantize` functions below.
2. Run the code on an image of your choosing. Display the results for a few different values of $b$.

> *Note 1: If you actually try saving this as a file, you won't see the file size being what you expected, because Python won't know to allocate exactly $b$ bits per element of `quantized_img`, and will instead probably store them as 32-bit integers if you don't specify otherwise. But if one wanted to work harder, it would be theoretically possible to store the elements with $b$ bits per pixel. Also, all this is before any additional lossless compression.*

> *Note 2: This is not how image compression systems like JPEG actually work. They use something similar to the Fourier transform, followed by a (simpler) quantization, followed by lossless compression.*

In [ ]:
import os

from matplotlib.pyplot import imread, imshow
from sklearn.cluster import KMeans

In [ ]:
# This is an  tuberose from our garden in Vancoue
img = imread(os.path.join("eva-happy-saturday.jpg"))
imshow(img)
plt.title("original image")
plt.show()

In [ ]:
def quantize(img, b):
    """
    Quantizes an image into 2^b clusters

    Parameters
    ----------
    img : a (H,W,3) numpy array
      the image to be processed
    b   : int
      the desired number of bits

    Returns
    -------
    quantized_img : a (H,W) numpy array containing cluster indices
    colours       : a (2^b, 3) numpy array, each row is a colour
    """

    H, W, _ = img.shape
    model = KMeans(n_clusters=2 ** b)

    ### YOUR CODE
    

    return quantized_img, model.cluster_centers_.astype("uint8")


def dequantize(quantized_img, colours):
    H, W = quantized_img.shape
    img = np.zeros((H, W, 3), dtype="uint8")

    # YOUR CODE: fill in the values of `img` here

    

    return img

In [ ]:
b = 3
compressed, colours = quantize(img, b=b)
recon = dequantize(compressed, colours)
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(img)
ax[0].set_title("original image")
ax[1].imshow(recon)
ax[1].set_title(f"reconstructed image (b = {b})");

In [ ]:
plt.imshow(colours[None])
plt.title("colours learned")
plt.xticks([])
plt.yticks([])
plt.show()

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** 

When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from "1" will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Push all your work to your GitHub lab repository. 
4. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 
5. Make sure that the plots and output are rendered properly in your submitted file. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb so that the TAs can view your submission on Gradescope. 

Well done!! Have a great weekend! 

In [ ]:
from IPython.display import Image

Image("eva-well-done.png")